In [41]:
import pandas as pd

In [42]:
items = pd.read_csv('data/items.csv')
fulldata = pd.read_csv('data/fulldata.csv')

In [43]:
fulldata['datetime'] = pd.to_datetime(fulldata['datetime'])
fulldata = fulldata.merge(items[['series_id', 'item_name']], on='series_id')
tilltoday = fulldata.query('datetime.dt.date == datetime.dt.date.max()')

In [44]:
# 3 methods for predicting the price of groceries over the next year
# 1. 3% growth rate as USDA predicts
# 2. continued average growth rate of last year
# 3. regression model based on historical data

In [45]:
# make a line graph of the price of the item over the last year

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [47]:
# generic application of compound interest formula for growth rates
# compund interest formula: A = P(1 + r/n)^(nt)

compound_interest = lambda P, r, n, t: P * (1 + r/n) ** (n * t)

def NextTwelve(list_of_prices, growth_rate=0.03):
    last_price = list_of_prices[-1]
    next_twelve_months = np.arange(1, 13)
    next_twelve_months = compound_interest(last_price, growth_rate, 1, next_twelve_months)
    return next_twelve_months

In [48]:
# 1. 3% growth rate as USDA predicts
usdarate = 0.066

# 2. continued average growth rate of last year
def AverageRate(list_of_prices, months=12):
    last_twelve_months = list_of_prices[-months:]
    average_rate = np.average(np.diff(last_twelve_months)/last_twelve_months[:-1])
    return average_rate

def USDAModel(list_of_prices, growth_rate=0.03):
    nt = NextTwelve(list_of_prices, growth_rate)
    return np.insert(nt, 0, list_of_prices[-1])

def AverageModel(list_of_prices):
    average_rate = AverageRate(list_of_prices, months=12)
    nt = NextTwelve(list_of_prices, average_rate)
    return np.insert(nt, 0, list_of_prices[-1])

In [79]:
from sklearn.linear_model import LinearRegression
# import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

In [113]:
# 3. regression model based on historical data

def RegressionModel(list_of_prices):
    X = np.arange(1, len(list_of_prices) + 1).reshape(-1, 1)
    y = np.array(list_of_prices).reshape(-1, 1)
    reg = RandomForestRegressor(max_depth=3, n_estimators=500, random_state=123)
    model = reg.fit(X, y)
    nt = model.predict(np.arange(len(list_of_prices) + 1, len(list_of_prices) + 13).reshape(-1, 1)).flatten()
    return np.insert(nt, 0, list_of_prices[-1])

In [114]:
predictable_items = items.query('item_name in {}'.format(tilltoday.item_name.to_list())).item_name.unique()
predictable_items = pd.DataFrame(predictable_items, columns=['item_name'])
predictable_items.to_csv('data/predictable_items.csv', index=False)

In [115]:
predictable_items.head()

,item_name
0,"Flour, white, all purpose, per lb. (453.6 gm)"
1,"Rice, white, long grain, uncooked, per lb. (45..."
2,"Spaghetti and macaroni, per lb. (453.6 gm)"
3,"Bread, white, pan, per lb. (453.6 gm)"
4,"Bread, whole wheat, pan, per lb. (453.6 gm)"


In [122]:
datapredictions = pd.DataFrame(columns=['item_name', 'datetime', 'usda grocery prediction', 'continued average', 'simple regression'])
for item in predictable_items['item_name']:
    itemdata = fulldata.query('item_name == "{}"'.format(item)).sort_values('datetime')
    next_12_months = pd.date_range(itemdata.datetime.max(), periods=13, freq='MS')
    datapredictions = pd.concat([datapredictions, pd.DataFrame({
        'item_name': [item]*13,
        'datetime': next_12_months,
        'usda grocery prediction': USDAModel(itemdata.value.values),
        'continued average': AverageModel(itemdata.value.values),
        'simple regression': RegressionModel(itemdata.value.values),
    })])

/tmp/ipykernel_337056/3190187276.py:7: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/tmp/ipykernel_337056/3190187276.py:7: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/tmp/ipykernel_337056/3190187276.py:7: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/tmp/ipykernel_337056/3190187276.py:7: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/tmp/ipykernel_337056/3190187276.py:7: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/tmp/ipykernel_33705

In [123]:
datapredictions.to_csv('data/datapredictions.csv', index=False)
datapredictions.head()

,item_name,datetime,usda grocery prediction,continued average,simple regression
0,"Flour, white, all purpose, per lb. (453.6 gm)",2023-03-01,0.546000,0.546000,0.546000
1,"Flour, white, all purpose, per lb. (453.6 gm)",2023-04-01,0.562380,0.555479,0.534458
2,"Flour, white, all purpose, per lb. (453.6 gm)",2023-05-01,0.579251,0.565123,0.534458
3,"Flour, white, all purpose, per lb. (453.6 gm)",2023-06-01,0.596629,0.574934,0.534458
4,"Flour, white, all purpose, per lb. (453.6 gm)",2023-07-01,0.614528,0.584915,0.534458


In [124]:
predictable_items['item_name'][40]

'All Uncooked Beef Steaks, per lb. (453.6 gm)'

In [125]:
item = predictable_items['item_name'][2]
start_date = '2022-04-01'
end_date = '2024-04-01'
dfa = fulldata.query("item_name == '{}'".format(item))
dfb = datapredictions.query("item_name == '{}'".format(item))
fig = px.line(dfb, x="datetime", y=['usda grocery prediction', 'continued average', 'simple regression'], title="Historical and Predicted Prices of {}".format(item), range_x=[start_date, end_date])
fig.add_scatter(x=dfa.datetime, y=dfa.value, mode='lines', name='actual')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=1,
    xanchor="left",
    x=0,
    title='Prediction Methods',
), xaxis_title='', yaxis_title=None, yaxis_tickprefix='$', yaxis_tickformat=',.2f', title_x=0.5, margin = dict(t=100, b=0, l=100, r=100)
)
fig.show()